In [1]:
# Tehdään koneoppimismalli bayes ja svc, joka pystyy määrittämään viinin laadun (variety) sen 
# kuvauksen perusteella (description)
# Kuvaus on luonnollista kieltä, eli normaalia tekstiä.
# Data on Kagglesta.
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
# Luetaan wine.csv dataframeen.
df = pd.read_csv('wine.csv')

In [76]:
df.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

In [29]:
df.shape

(71322, 14)

In [5]:
# Lasketaan erilaiset laadut ja otetaan niistä top10.
# filteröidään mukaan vain ne jotka kuuluvat tähän top10 varietyyn.
counter = Counter(df['variety'].tolist())
top_10_varieties = {i[0]: idx for idx, i in enumerate(counter.most_common(10))}
df = df[df['variety'].map(lambda x: x in top_10_varieties)]

In [50]:
# Tehdään lista kuvauksista ja tehdään lista viinilaaduista.
# Tehdään np.array vektori jossa laadun paikalla on sijoitus top10 listalla.
description_list = df['description'].tolist()
varietal_list = [top_10_varieties[i] for i in df['variety'].tolist()]
varietal_list = np.array(varietal_list)

In [63]:
df['variety'].head(10).tolist()

['Riesling',
 'Pinot Noir',
 'Cabernet Sauvignon',
 'Cabernet Sauvignon',
 'Chardonnay',
 'Riesling',
 'Red Blend',
 'Pinot Noir',
 'Merlot',
 'Pinot Noir']

In [64]:
varietal_list = [top_10_varieties[i] for i in df['variety'].head(10).tolist()]

In [66]:
top_10_varieties

{'Bordeaux-style Red Blend': 4,
 'Cabernet Sauvignon': 2,
 'Chardonnay': 1,
 'Merlot': 9,
 'Pinot Noir': 0,
 'Red Blend': 3,
 'Riesling': 5,
 'Rosé': 8,
 'Sauvignon Blanc': 6,
 'Syrah': 7}

In [65]:
varietal_list

[5, 0, 2, 2, 1, 5, 3, 0, 9, 0]

In [60]:
len(varietal_list)

71322

In [55]:
np.array(varietal_list)

array([5, 0, 2, ..., 2, 5, 0])

In [7]:
# Muutetaan kuvaukset tekstistä harvaksi lukumatriisiksi.
count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(description_list)

In [8]:
# Muutetaan lukumatriisi normalisoituun TF-IDF -muotoon.
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

In [9]:
# Splitataan train ja test set tehdystä matriisista ja vektorista.
train_x, test_x, train_y, test_y = train_test_split(x_train_tfidf, varietal_list, test_size=0.3)

In [16]:
# Tehdään malli: bayes tai svc. SVC on tarkempi mutta ajo kestää useita minuutteja.
#clf = MultinomialNB().fit(train_x, train_y)
from sklearn.svm import SVC
clf = SVC(kernel='linear').fit(train_x, train_y)
# Mallin perusteella tehdään ennusteet matriisiin.
y_score = clf.predict(test_x)

In [17]:
# Lasketaan montako meni oikein ja saadaan laskettua mallin tarkkuus.
# Montako samaa saadaan y_score ja test_y?
n_right = 0
for i in range(len(y_score)):
    if y_score[i] == test_y[i]:
        n_right += 1

In [18]:
# Jaetaan löytyneiden lukumäärä koko test-vektorin pituudella ja kerrotaan sadalla 
# jolloin saadaan laskettua mallin tarkkuuus.
print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

Accuracy: 80.38%
